# SB Code Review
[X] Audio Embeddings Manager  
[ ] Bayes Search  
[ ] Librosa Manager  
[ ] Audio Manager  
[ ] Saved Feature   
[ ] FullPipelineExamjple-RB-Edit  

In [1]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
from packages.LJDataLoader import LJDataLoader

/home/ubuntu/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-05-30 09:49:59.950918: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 09:50:00.057725: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-30 09:50:00.601807: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.c

In [7]:
def generate_split(fake_cols, file_path):

    
    loader = LJDataLoader(data_path=file_path)
    loader.sample(0.1)
    loader.splitData()

    source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
    new_col_name = 'RandWaveFake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)
    
    source_architectures = ['RandWaveFake', 'ElevenLabs', 'UberDuck']
    new_col_name = 'Fake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

    #data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['RandWaveFake', 'ElevenLabs', 'UberDuck'])
    data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=fake_cols)
    
    train_count = data_df[data_df['type'] == 'train'].shape[0]
    dev_count = data_df[data_df['type'] == 'dev'].shape[0]
    test_count = data_df[data_df['type'] == 'test'].shape[0]

    print(f'# of Train instances: {train_count}')
    print(f'# of Dev instances: {dev_count}')
    print(f'# of Test instances: {test_count}')
    
    return data_df

df = generate_split(['ElevenLabs', 'UberDuck', 'Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow'], '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv')
df.head()

# of Train instances: 7830
# of Dev instances: 2610
# of Test instances: 2620


,type,id,architecture,path,label,multiclass_label
0,train,LJ046-0200,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,LJ027-0027,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,LJ008-0280,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,LJ011-0214,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,LJ041-0167,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0


# Audio Embeddings Manager

In [2]:
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager

In [17]:
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=df)

[NeMo I 2023-05-30 10:13:02 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-05-30 10:13:02 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-05-30 10:13:02 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-05-30 10:13:03 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-05-30 10:13:03 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-05-30 10:13:03 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-05-30 10:13:03 features:267] PADDING: 16
[NeMo I 2023-05-30 10:13:03 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


In [18]:
test_embeddings_df, test_embeddings_cols = embedding_manager.generateFeatureDf()

100%|██████████| 10/10 [00:02<00:00,  3.69it/s]


In [19]:
test_embeddings_df

,type,id,architecture,path,label,multiclass_label,0,1,2,3,...,182,183,184,185,186,187,188,189,190,191
0,train,LJ046-0200,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,-0.011599,0.020590,0.007400,0.030296,...,0.004632,-0.015568,0.010481,-0.022734,-0.003763,0.011702,-0.011795,0.014943,0.010467,-0.005011
1,train,LJ027-0027,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,-0.007376,0.012096,0.000776,-0.001059,...,0.006214,0.021153,0.001134,-0.018028,0.003293,-0.000954,0.003614,0.015869,0.004958,-0.022131
2,train,LJ008-0280,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,-0.003211,-0.003519,-0.002119,0.014034,...,-0.002582,-0.001631,-0.022860,0.006669,-0.002203,0.026715,0.002739,-0.003260,0.003903,-0.004147
3,train,LJ011-0214,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,-0.004525,0.011935,0.006839,0.019599,...,-0.000705,-0.002297,-0.000625,-0.034258,0.003794,0.023903,0.003146,0.015346,0.021568,-0.004405
4,train,LJ041-0167,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,0.007247,0.010398,-0.012832,0.026711,...,-0.001727,0.000518,-0.007738,-0.006469,-0.002472,0.030121,-0.011498,0.021411,0.002436,-0.007514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13055,test,LJ036-0023,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9,0.001503,-0.012607,0.002449,0.054512,...,-0.000631,0.001209,-0.009306,-0.009831,-0.009804,0.016721,0.001947,0.027883,0.012046,-0.001425
13056,test,LJ040-0180,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9,-0.015063,-0.014280,0.018317,0.035471,...,-0.013205,-0.017616,-0.018896,0.000753,-0.005224,0.021934,0.001776,0.005952,0.002285,-0.020610
13057,test,LJ008-0294,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9,-0.012186,0.014306,0.000118,0.024648,...,-0.013042,-0.008875,-0.033794,-0.002515,-0.030894,0.031642,-0.001458,0.002432,0.002236,-0.021676
13058,test,LJ002-0292,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9,-0.008180,-0.004592,0.025996,0.031908,...,-0.010992,-0.015574,-0.009301,-0.006878,-0.009143,0.024405,0.000539,0.013197,-0.003473,-0.012696


# Librosa manager

In [5]:
#from packages.LibrosaManager import LibrosaManager
from json import load
import librosa

In [28]:
class LibrosaManager:

    def __init__(self, audio_path):

        assert not isinstance(audio_path, type(None)), 'Must provide a valid audio path'
        
        self.time_series = None
        self.sampling_rate = None 
        self._loadAudio(audio_path)
        self.duration = librosa.get_duration(y=self.time_series)

        self.mfcc = None
        
    def _loadAudio(self, audio_path):
        self.time_series, self.sampling_rate = librosa.load(audio_path)

    def generateMFCC(self, n_mfcc, window_time, hop_time, load_object=False):

        win_length = int(self.sample_rate/1000 * window_time)
        hop_length = int(self.sample_rate/1000 * hop_time)

        mfcc =  librosa.feature.mfcc(self.time_series, n_mfcc=n_mfcc, win_length=win_length, hop_length=hop_length)

        if load_object:
            self.mfcc = mfcc

        return mfcc

    def resample(self, target_sample_rate):
        return librosa.resample(y=self.time_series.T, orig_sr=self.sampling_rate, target_sr=target_sample_rate)


In [14]:
# Tesing assertion error 
librosa_audio = LibrosaManager('../../data/wavefake_data/LJSpeech_1.1/wavs/16000KHz/LJ001-0006.wav')

In [18]:
librosa_audio.time_series

array([ 7.8725694e-05,  1.6993821e-04,  1.9835136e-04, ...,
        7.8591210e-04,  2.9640517e-04, -1.2645208e-04], dtype=float32)

In [19]:
librosa.load('../../data/wavefake_data/LJSpeech_1.1/wavs/16000KHz/LJ001-0006.wav')

(array([ 7.8725694e-05,  1.6993821e-04,  1.9835136e-04, ...,
         7.8591210e-04,  2.9640517e-04, -1.2645208e-04], dtype=float32),
 22050)

## To discuss: sample rate
https://librosa.org/doc/main/generated/librosa.load.html - DEFAULT VS. 'SR = NONE'

librosa.load(path, *, sr=22050, mono=True, offset=0.0, duration=None, dtype=<class 'numpy.float32'>, res_type='soxr_hq')[source]¶
Load an audio file as a floating point time series.

Audio will be automatically resampled to the given rate (default sr=22050).

To preserve the native sampling rate of the file, use sr=None.

<b>Checking resapling behaviour</b>
What is resampling to librosa? we hav two possible definitoons:

1. overwrite the files 'sample rate' so that it plays at a new sample rate when opneed
2. Physically (statsitically) resample the observations up or down- like tradiitonal time series 

In [23]:
import numpy as np
import matplotlib.pyplot as plt
# sphinx_gallery_thumbnail_path = '_static/playback-thumbnail.png'

import librosa

# We'll need IPython.display's Audio widget
from IPython.display import Audio

# Testing the default
audio = librosa.load('../../data/wavefake_data/LJSpeech_1.1/wavs/Original/LJ001-0003.wav')
print(f'Sample rate: {audio[1]}')
Audio(data=audio[0], rate=audio[1])

# Testing a higher SR one
audio = librosa.load('../../data/wavefake_data/LJSpeech_1.1/wavs/Original/LJ001-0003.wav', sr=44100)
print(f'Sample rate: {audio[1]}')
Audio(data=audio[0], rate=audio[1])

Sample rate: 22050
Sample rate: 44100


In [25]:
# Testing a lower SR 
audio = librosa.load('../../data/wavefake_data/LJSpeech_1.1/wavs/Original/LJ001-0003.wav', sr=3000)
print(f'Sample rate: {audio[1]}')
Audio(data=audio[0], rate=audio[1])

Sample rate: 3000


In [26]:
# Testing an extremely high SR 
audio = librosa.load('../../data/wavefake_data/LJSpeech_1.1/wavs/Original/LJ001-0003.wav', sr=100000)
print(f'Sample rate: {audio[1]}')
Audio(data=audio[0], rate=audio[1])

Sample rate: 100000


In [ ]:
# Trying resampling
audio = librosa.load('../../data/wavefake_data/LJSpeech_1.1/wavs/Original/LJ001-0003.wav', sr=None, res_type=)
print(f'Sample rate: {audio[1]}')
Audio(data=audio[0], rate=audio[1])

## Findings
* You can resample on ingestion- https://librosa.org/doc/main/generated/librosa.load.html

'# Load a file and resample to 11 KHz
filename = librosa.ex('trumpet')
y, sr = librosa.load(filename, sr=11025)
y
array([-8.746e-04, -3.363e-04, ..., -1.301e-05,  0.000e+00],
      dtype=float32)
sr
11025'
* <font color='red'>Basically - we are already resampling because we dont specify 'sr=None'. 

# AudioManager

In [29]:
from pydub import AudioSegment
import os
import soundfile as sf
import librosa
import numpy as np
import random 
import shutil

class AudioManager:

    def __init__(self) -> None:
        pass

    def convertAudioDirectory(self,
                                audio_dir: str,
                                input_format: str,
                                output_format: str = '.wav',
                                output_dir: str = None,
                                delete_original: bool = False,
                                bitrate: str = None,
                                codec: str = None):

        for file in os.listdir(audio_dir):

            if input_format in file:
                self.convertAudioFileTypes(os.path.join(audio_dir, file), output_format=output_format, 
                                            delete_original=delete_original, output_dir=output_dir,
                                            bitrate=bitrate, codec=codec)
                


    def convertAudioFileTypes(self, audio_path: str,
                                output_format: str = '.wav',
                                delete_original: bool = False,
                                output_dir: str = None,
                                output_file_name: str = None,
                                bitrate: str = None,
                                codec: str = None):
        
        assert output_format in ['.wav', '.mp4'], f'{output_format} is an invalid output format. Please enter types: (.wav, .mp4).'
        
        try:
            import_audio = AudioSegment.from_file(audio_path)

            if isinstance(output_file_name, type(None)):
                output_file_name = os.path.basename(audio_path)
            output_file_name = output_file_name.replace(os.path.splitext(output_file_name)[1], output_format)

            if not output_dir:
                output_dir = os.path.dirname(audio_path)

            import_audio.export(os.path.join(output_dir, output_file_name),
                                format=output_format.replace('.', ''),
                                codec=codec,
                                bitrate=bitrate)

            if delete_original:
                os.remove(audio_path)
                
        except Exception as e:
            print(f'Failed to Convert Audio File: {audio_path}')
            print('Error: ', e)

    def resampleAudioDirectory(self, input_directory: str, output_directory: str, target_sample_rate: int, replace_existing: bool = False):
        
        for file in os.listdir(input_directory):
            
            if os.path.splitext(file)[1] not in ['.wav', '.mp4']:
                continue
            
            if not replace_existing:
                if os.path.isfile(os.path.join(output_directory, file)):
                    continue
            
            try:
                librosa_manager = LibrosaManager(os.path.join(input_directory, file))
                resampled_audio = librosa_manager.resample(target_sample_rate)
                sf.write(os.path.join(output_directory, file), resampled_audio, target_sample_rate, subtype='PCM_24')
            except Exception as e:
                print(f'Failed to Resample: {file}')
                print(f'Error Msg: {e}')
                print()
                
    
    def addNoiseWithSnr(self, audio_path: str, snr_range: list = [10, 80]):
        
        audio, sr = librosa.load(audio_path)
        
        audio_power = np.mean(audio ** 2)
        
        noise_snr = random.randint(snr_range[0], snr_range[1])
        noise_power = audio_power / (10 ** (noise_snr / 10))
        noise = np.random.normal(scale=np.sqrt(noise_power)*100, size=len(audio))

        noisy_audio = audio + noise

        return noisy_audio, noise_snr, sr

    def launderAudioDirectory(self, input_dir: str, output_dir: str, noise_type: str = 'random_gaussian', replace_existing: bool = False, transcode_prob=0.5, noise_prob=0.5):

        full_launder_details = []

        # Loop through files 
        for file in os.listdir(input_dir):

            #[is_transcode, bitrate, is_noise, snr]
            file_launder_details = [os.path.join(input_dir, file), 0, None, 0, None]


            #if not replace_existing:
            #    if os.path.isfile(os.path.join(output_dir, file)):
            #        continue

            try:
                is_transcode = np.random.rand() <= transcode_prob
                is_noise = np.random.rand() <= noise_prob

                bitrate_options = ['64k', '127k', '196k']


                if is_transcode:

                    bitrate = random.choice(bitrate_options)

                    file_launder_details[1] = 1
                    file_launder_details[2] = bitrate

                    self.convertAudioFileTypes(os.path.join(input_dir, file), output_dir=output_dir,
                                             output_format='.mp4', delete_original=False, bitrate=bitrate,
                                             codec='aac'
                                            )

                    self.convertAudioFileTypes(os.path.join(output_dir, file.replace('wav', 'mp4')), output_format='.wav', delete_original=True)

                else:
                    # if no transcode is necessary, just move the file to the new directory
                    shutil.copy(os.path.join(input_dir, file), os.path.join(output_dir, file))

                if is_noise:

                    noisy_audio, noise_snr, sr = self.addNoiseWithSnr(os.path.join(output_dir, file))

                    file_launder_details[3] = 1
                    file_launder_details[4] = noise_snr


                    #sf.write(os.path.join(output_dir, file), noisy_audio, target_sample_rate, subtype='PCM_24')
                    sf.write(os.path.join(output_dir, file), noisy_audio, sr)

                full_launder_details.append(file_launder_details)

            except Exception as e:
                print(f'Failed to add noise: {file}')
                print(f'Error Msg: {e}')
                print()

        return full_launder_details


In [31]:
# Testing assertion errors
audio_man = AudioManager()
audio_man.convertAudioDirectory(audio_dir='../../data/wavefake_data/LJSpeech_1.1/wavs/Original/', input_format='.mp4', output_format= '.hello')

# Bayes search

In [ ]:
# TBC

# For paper

In [47]:
# N unique speakers and phrases
speakers = []
phrases = []

for item in os.listdir('../../data/TIMIT converted/wav_files'):
    speakers.append(item.split('_')[0])
    phrases.append(item.split('_')[1].split('.')[0])
    
print(len(set(speakers)))
print(len(set(phrases)))

462
1718


In [46]:
# N Fake voices (both built in and cloned)
for item in os.listdir('../../data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs//SI2237/fake'):
    print(item)

Sam_SI2237.wav
Elli_SI2237.wav
Obama_SI2237.wav
Josh_SI2237.wav
Rachel_SI2237.wav
Adam_SI2237.wav
Domi_SI2237.wav
Antoni_SI2237.wav
Biden_SI2237.wav
Bella_SI2237.wav
Arnold_SI2237.wav


In [52]:
# N voices per speaker
speakers = []
phrases = []

for item in os.listdir('../../data/TIMIT converted/wav_files'):
    speakers.append(item.split('_')[0])
    phrases.append(item.split('_')[1].split('.')[0])
    
speaker_counts = [speakers.count(item) for item in set(speakers)]
phrase_counts = [phrases.count(item) for item in set(phrases)]
print(len(set(speakers)))
print(len(set(phrases)))

462
1718


In [59]:
len([i for i in phrase_counts if i > 1])

332